In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from itertools import product, combinations
import numpy as np
import scipy
from scipy.spatial.transform import Rotation as R
from scipy.optimize import minimize, basinhopping
import matplotlib
matplotlib.rcParams['figure.dpi'] = 140
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
from tqdm import tqdm
import shutil
import pickle

# Geometric Objects:

## Abstract superclass:

In [2]:
class GeometricObject:
    """Abstract superclass defining a generic 3D geometric object with location, rotation, and scale.

    A GeometricObject is defined by its location in global cooridnates, rotation in Euler angles,
    and scale (all objects share the same scale). For all objects, location and z-rotation are 
    treated as optimizable, and scale is treated as fixed. To define a specific type of geometric 
    object (see class Cuboid below), the methods get_bounding_intervals and add_self_to_axis must be
    implemented. Sub-classes may have additional attributes.

    Attributes:
        loc: a length 3 np.array of form [loc_x, loc_y, loc_z].
        rot: a length 3 np.array of form [rot_x, rot_y, rot_z].
        scale: a length 3 np.array of form [scale_x, scale_y, scale_z].
    """
    def __init__(self, loc, rot, scale):
        """Init method.
        
        Args: 
            loc: list-like with form [X,Y,Z], object centroid coordinates.
            rot: list-like with form [X,Y,Z], object rotation in degrees.
            scale: list-like with form [X,Y,Z], object scale.
        Returns:
            None
        """
        self.loc = np.array(loc)
        self.rot = np.array(rot)
        self.scale = np.array(scale)

    def get_all_attr(self):
        """Get all attributes in the form of a flat np.array.
        
        Args:
            None.
        Returns:
            A flat np.array with every optimizable attribute stacked together.
            It looks like [loc_x,loc_y,loc_z,rot_x,rot_y,rot_z,scale_x,scale_y,scale_z].
        """
        return np.concatenate([self.loc, self.rot, self.scale])

    def get_optimizable_attr(self):
        """Get the attributes that can be optimized in the form of a flat np.array.
        
        Generally, we assume location and z-rotation to be optimizable, and scale to be fixed.
        x and y rotation are assumed to be fixed.
        
        Args:
            None.
        Returns:
            A flat np.array with every optimizable attribute stacked together.
            It looks like [loc_x,loc_y,loc_z,rot_z].
        """
        return np.concatenate([self.loc, self.rot[2:3]])

    def set_optimizable_attr(self, attr_array):
        """Set optimizable attributes with an np.array of the same form as one from get_optimizable_attr.

        The purpose of this method is to quickly convert an array generated by get_optimizable_attr and
        optimized by the scipy optimizer back to self.geometry.

        Args:
            attr_array: Flat np.array of same form as one returned by get_optimizable_attr.
        Returns:
            None. Modifies self.geometry.
        """
        self.loc = attr_array[:3]
        self.rot[2] = attr_array[3]

    def get_bounding_intervals(self):
        """Get the largest and lowest coordinate values along each dimension of the object as intervals.

        The purpose of this method is to easily get a "bounding box" in the form of three (along
        the X, Y, and Z axes) intervals that fully (and ideally tightly) bound the range of points
        spanned by the object. This way, regardless of the object's shape or orientation, we can
        get a cuboid bound that is always parallel to the global coordinate axes. Having this method
        makes constraints that rely on having the "leftmost" or "topmost" points aligned easier to write.

        Args:
            None.   
        Returns:
            A tuple of three lists ([x_min, x_max], [y_min, y_max], [z_min, z_max]) that bounds the object.
        """
        raise NotImplementedError

    def add_self_to_axis(self, ax, label, color):
        """Add self as a 3D wireframe shape with appropriate loc/rot/scale to an matplotlib axis.

        Args:
            ax: A matplotlib axis with projection set to 3D.
            label: A str, label to display in legend.
            color: A valid matplotlib color to use for this object's wireframe.
        Returns:
            A color patch with label to be passed to legend handler. Also modifies the input argument ax.
        """
        raise NotImplementedError

## Concrete objects:

In [3]:
class Cuboid(GeometricObject):
    """A cuboid.

    Inherits from the GeometricObject abstract class. A cuboid is a unit cube by default,
    the location/rotation/size of which is determined by the object's loc/rot/scale.

    Attributes:
        Same as super-class, no additional attributes. Add additional attributes as needed.
    """
    def __init__(self, loc, rot, scale):
        super().__init__(loc, rot, scale)

    def get_bounding_intervals(self):
        """Get bounding intervals around the (possibly rotated) cuboid.

        See super-class documentation for specifics. Note that, for a cuboid, the bounding
        intervals will be perfectly tight if the cuboid is not rotated/rotated with right
        angles, but otherwise these intervals will be loose.
        """
        # get the corners first -> extreme points can only be corners
        corners = self.get_corners()

        # find the max and min along each dimension
        min_by_dim = corners.min(axis=0)
        max_by_dim = corners.max(axis=0)

        # collect into tuple of three lists and return
        return ([min_by_dim[0], max_by_dim[0]],
                [min_by_dim[1], max_by_dim[1]],
                [min_by_dim[2], max_by_dim[2]])
        
    def add_self_to_axis(self, ax, label, color):
        """Add self as 3D wireframe cuboid to ax.

        See super-class documentation for specifics. For a cuboid, we just draw a simple
        wireframe connecting all vertices.
        """
        # get the corners to draw wireframe on
        vertices = self.get_corners()

        # # generate all pairwise combinations of vertices for wireframe and plot only edges
        # for start, end in combinations(vertices, 2):
        #     l2_dist = np.linalg.norm(start - end)
        #     if (abs(l2_dist-self.scale_x) < 1e-4        # only plot the lines that are edges
        #         or abs(l2_dist-self.scale_y) < 1e-4     # identify edges if distance is close to scale
        #         or abs(l2_dist-self.scale_z) < 1e-4):   # 1e-4 tolerance added due to numerical inpercision
        #         ax.plot(*zip(start,end), color=color)

        # the above works, but does needless computation and is subject to numerical lack of precision
        # more direct method: just plot all twelve lines directly since get_corners returns corners in fixed order
        ax.plot(*zip(vertices[0],vertices[1]), color=color) # lower rectangle
        ax.plot(*zip(vertices[1],vertices[3]), color=color)
        ax.plot(*zip(vertices[3],vertices[2]), color=color)
        ax.plot(*zip(vertices[2],vertices[0]), color=color)
        ax.plot(*zip(vertices[4],vertices[5]), color=color) # upper rectangle
        ax.plot(*zip(vertices[5],vertices[7]), color=color)
        ax.plot(*zip(vertices[7],vertices[6]), color=color)
        ax.plot(*zip(vertices[6],vertices[4]), color=color)
        ax.plot(*zip(vertices[0],vertices[4]), color=color) # connecting upper and lower rectangles
        ax.plot(*zip(vertices[1],vertices[5]), color=color)
        ax.plot(*zip(vertices[2],vertices[6]), color=color)
        ax.plot(*zip(vertices[3],vertices[7]), color=color)

        # return a patch for legend
        return mpatches.Patch(color=color, label=label)
        

    def get_corners(self):
        """Get the corner vertices of the cuboid.

        Cuboid-specific helper method to faciliate plotting and bounding-interval finding.

        Args:
            None
        Returns:
            An np.array of shape (8,3) with all eight corners of the cuboid.
            NOTE: The corners are returned in default order of itertools.product([-0.5,0.5], [-0.5,0.5], [-0.5,0.5]).
        """
        # define base (unit-cube) corners
        corners = np.array(list(product([-0.5,0.5], [-0.5,0.5], [-0.5,0.5])))

        # get rotation matrix from Euler angles
        rotation_matrix = R.from_euler(angles=self.rot,
                                       seq="xyz", degrees=True).as_matrix()

        # transform base corners with loc, rot, and scale
        corners = corners * self.scale # scale
        corners = corners @ rotation_matrix # rotate
        corners = corners + self.loc # translate
        return corners

# Constraints:

## Abstract superclass:

In [4]:
class ConstraintProposition:
    """Abstract superclass representing a proposition asserting the satisfaction of a constraint between given objects.

    All constraints should inherit from this class. A constraint proposition is a proposition asserting
    that some constraint (defined by the badness method) is satisfied on the given set of objects
    passed at initialization (stored in self.arguments) -- for example, given a sub-class IsAligned, 
    the initialization of that sub-class might look like IsAligned((cube1, cube2), kwargs) and represent
    the proposition that cube1 and cube2 are aligned. To define a new constraint, one only needs to
    implement the badness method (how badly is the constraint violated?) and the define_arity method (what is
    the arity of this proposition?).

    Attributes:
        arity: int or None, the arity of this proposition. None denotes flexible arity.
        arguments: list (length equals arity) of objects of class GeometricObject, the arguments to constrain.
    """
    def __init__(self, arguments):
        """Init method.
        
        Args: 
            arguments: A list-like of Object objects.   
        Returns:
            None
        Raises:
            ValueError if there is an arity mismatch.
        """
        self.arity = self.define_arity() # get the arity
        arguments = list(arguments) # ensure arguments is a list

        # check to make sure list has correct arity
        if self.arity is not None and len(arguments) != self.arity:
            raise ValueError(f"Input arguments has arity {len(arguments)}, but constraint was defined with arity {self.arity}.")

        # if it does, store as attribute
        self.arguments = arguments

    def define_arity(self):
        """Define the arity of the constraint proposition, i.e. length of self.arguments.

        THIS METHOD SHOULD RETURN EITHER AN INTEGER OR NONE. NONE denotes flexible
        arity (the badness method will, in that case, have to be written accordingly
        to process a whole list and cannot make assumptions about its length).

        Args:
            None.
        Returns:
            An integer or None denoting arity.
        """
        raise NotImplementedError
        
    def badness(self):
        """Compute the badness of satisfaction of this constraint.
        
        Each type of constraint will have a different definition for badness of satisfaction
        and so will each need to implement this method separately. Furthermore, this method
        only operates on attributes and must take no additional arguments. Any information
        other than self.arguments should be stored as additional attributes by __init__.
        
        Args:
            None. 
        Returns:
            A scalar float value indicating badness of satisfaction of this constraint by self.arguments.
        """
        raise NotImplementedError

## Primitive (base) constraints:

In [5]:
class HasLocation_U(ConstraintProposition):
    """A proposition that asserts that an object has a fixed location.

    NOTE: This constraint proposition is unary ("_U").

    Inherits from the Object abstract class. The target location this constraint proposition
    asserts the object to have is defined by a 3-element list or list-like such as [10,5,None].
    The first element is assumed to be for the x dimension, the second for the y, and the third
    for the z. If any of the entries are None, then that dimension is not constrained.

    Attribtues:
        Superclass attributes.
        target_loc: list of 3 elements of form [x,y,z].
    """
    def __init__(self, arguments, target_loc):
        """Init method."""
        super().__init__(arguments)

        # check to make sure target_loc is valid and store
        if len(target_loc) != 3:
            raise ValueError(f"Argument 'target_loc' must have length 3.")
        self.target_loc = target_loc

    def define_arity(self):
        """See superclass documentation for details."""
        return 1 # unary

    def badness(self):
        """See superclass documentation for details.
        
        In this case, badness is calculated as how far the the current location is
        from the target location. If any dimension of self.target_loc is None, that
        dimension is not factored into the calculations.
        """
        loc = self.arguments[0].loc # unary
        total_badness = 0
        if self.target_loc[0] is not None: total_badness += abs(self.target_loc[0] - loc[0])
        if self.target_loc[1] is not None: total_badness += abs(self.target_loc[1] - loc[1])
        if self.target_loc[2] is not None: total_badness += abs(self.target_loc[2] - loc[2])
        return total_badness

class AreTranslationallyAligned_F(ConstraintProposition):
    """A proposition that represents a notion of translational alignment between objects.

    NOTE: This constraint proposition is flexible-arity. It can be instantiated with any
          number of arguments. The output is simply the mean absolute deviation along the
          specified dimension at the specified location.

    Inherits from the Object abstract class. Alignment is defined by the self.dimension and
    self.location attriutes. The former represents the dimension along which alignment should be
    considered, and the latter represents which part of the objects one wants aligned (the
    left-most point? The center? Etc.). For example, if one wishes to articulate the constraint
    that "the top-most points on the given objects are aligned", one would set dimension="z"
    and location="bounding_max". If one wants to have the objects with centers aligned on the
    X-axis, one would set dimension="x" and location="center".

    Attributes:
        Superclass attributes.
        dimension: str, one of "x", "y", or "z", the dimension on which to assert alignment.
        location: str, one of "bounding_min", "bounding_max", or "center", where to check alignment.
    """
    def __init__(self, arguments, dimension, location):
        """Init method."""
        super().__init__(arguments)

        # check to make sure dimension and location are valid
        if dimension not in ["x", "y", "z"]:
            raise ValueError(f"Argument 'dimension' must be one of 'x', 'y', or 'z'.")
        if location not in ["bounding_min", "bounding_max", "center"]:
            raise ValueError(f"Argument 'dimension' must be one of 'bounding_min', 'bounding_max', or 'center'.")

        # store attributes
        self.dimension = dimension
        self.location = location

    def define_arity(self):
        """See superclass documentation for details."""
        return None # flexible-arity

    def badness(self):
        """See superclass documentation for details.
        
        In this case, badness is calculated as the mean absolute deviation between all objects
        passed along the specified dimension and location.
        """
        # convert str dimension to index
        dim_idx = 0
        if self.dimension == "x": dim_idx = 0
        elif self.dimension == "y": dim_idx = 1
        elif self.dimension == "z": dim_idx = 2

        # get values to compare for all objects
        values_to_compare = []
        for argument in self.arguments:
            if self.location == "bounding_min":
                values_to_compare.append(argument.get_bounding_intervals()[dim_idx][0])
            elif self.location == "bounding_max":
                values_to_compare.append(argument.get_bounding_intervals()[dim_idx][1])
            elif self.location == "center":
                values_to_compare.append(argument.loc[dim_idx])

        # take the mean absolute deviation and output
        values_to_compare = np.array(values_to_compare)
        return np.abs(values_to_compare - values_to_compare.mean()).mean()

class HaveTranslationalDifference_B(ConstraintProposition):
    """A proposition that represents a notion of translational difference between TWO objects.

    NOTE: This constraint proposition is binary ("_B"). This is because the notion of "difference" is
          not readily extendable to multiple objects. If one wants to achieve the effect of having
          two groups that are aligned within themselves and differ from each other by a certain amount,
          use two AreTranslationallyAligned constraints within both groups and one HaveTranslationalDifference_B
          constraint between two given objects one from each group.

    Inherits from the Object abstract class. Translational difference is defined according to the
    same self.dimension/self.location specification as the AreTranslationallyAligned constraint,
    but only operates over two objects and allows the user to specify how far one wants the objects
    apart with the self.target_difference attribute.

    Attributes:
        Superclass attributes.
        target_difference: float, the target difference in the specified location between argument 1 and 2.
        dimension: str, one of "x", "y", or "z", the dimension on which to assert alignment.
        location: str, one of "bounding_min", "bounding_max", or "center", where to check alignment.
    """
    def __init__(self, arguments, target_difference, dimension, location):
        """Init method."""
        super().__init__(arguments)

        # check to make sure dimension and location are valid
        if dimension not in ["x", "y", "z"]:
            raise ValueError(f"Argument 'dimension' must be one of 'x', 'y', or 'z'.")
        if location not in ["bounding_min", "bounding_max", "center"]:
            raise ValueError(f"Argument 'dimension' must be one of 'bounding_min', 'bounding_max', or 'center'.")

        # store attributes
        self.target_difference = target_difference
        self.dimension = dimension
        self.location = location

    def define_arity(self):
        """See superclass documentation for details."""
        return 2 # binary

    def badness(self):
        """See superclass documentation for details.
        
        In this case, badness is calculated as the difference between the target and actual differences
        between object 1 and object 2.
        """
        # convert str dimension to index
        dim_idx = 0
        if self.dimension == "x": dim_idx = 0
        elif self.dimension == "y": dim_idx = 1
        elif self.dimension == "z": dim_idx = 2

        # get values to compare for both objects (binary so only two arguments)
        values_to_compare = []
        for argument in self.arguments:
            if self.location == "bounding_min":
                values_to_compare.append(argument.get_bounding_intervals()[dim_idx][0])
            elif self.location == "bounding_max":
                values_to_compare.append(argument.get_bounding_intervals()[dim_idx][1])
            elif self.location == "center":
                values_to_compare.append(argument.loc[dim_idx])

        # take differences and output (binary so only two arguments)
        actual_difference = values_to_compare[0] - values_to_compare[1]
        return abs(self.target_difference - actual_difference)

class HasRotation_U(ConstraintProposition):
    """A proposition that asserts that an object has a fixed rotation.

    NOTE: This constraint proposition is unary ("_U").

    Inherits from the Object abstract class. The target rotation this constraint proposition
    asserts the object to have is defined by a 3-element list or list-like such as [30,None,45].
    The first element is assumed to be for the x dimension, the second for the y, and the third
    for the z. If any of the entries are None, then that dimension is not constrained.

    Attribtues:
        Superclass attributes.
        target_rot: list of 3 elements of form [x,y,z].
    """
    def __init__(self, arguments, target_rot):
        """Init method."""
        super().__init__(arguments)

        # check to make sure target_rot is valid and store
        if len(target_rot) != 3:
            raise ValueError(f"Argument 'target_rot' must have length 3.")
        self.target_rot = target_rot

    def define_arity(self):
        """See superclass documentation for details."""
        return 1 # unary

    def badness(self):
        """See superclass documentation for details.
        
        In this case, badness is calculated as how far the the current rotation is
        from the target rotation. If any dimension of self.target_rot is None, that
        dimension is not factored into the calculations.
        """
        rot = self.arguments[0].rot # unary
        total_badness = 0
        if self.target_rot[0] is not None: total_badness += abs(self.target_rot[0] - rot[0])
        if self.target_rot[1] is not None: total_badness += abs(self.target_rot[1] - rot[1])
        if self.target_rot[2] is not None: total_badness += abs(self.target_rot[2] - rot[2])
        return total_badness

class AreRotationallyAligned_F(ConstraintProposition):
    """A proposition that represents a notion of rotational alignment between objects.

    NOTE: This constraint proposition is flexible-arity. It can be instantiated with any
          number of arguments.

    Inherits from the Object abstract class. Alignment is defined by the self.dimension attribute.
    This constraint's badness is simply the mean absolute deivation of the angles (for specified
    dimension) of all argument objects.

    Attributes:
        Superclass attributes.
        dimension: str, one of "x", "y", or "z", the dimension on which to assert alignment.
    """
    def __init__(self, arguments, dimension):
        """Init method."""
        super().__init__(arguments)

        # check to make sure dimension and location are valid
        if dimension not in ["x", "y", "z"]:
            raise ValueError(f"Argument 'dimension' must be one of 'x', 'y', or 'z'.")

        # store attributes
        self.dimension = dimension

    def define_arity(self):
        """See superclass documentation for details."""
        return None # flexible-arity

    def badness(self):
        """See superclass documentation for details.
        
        In this case, badness is calculated as the mean absolute deviation between all objects
        passed along the specified dimension.
        """
        # convert str dimension to index
        dim_idx = 0
        if self.dimension == "x": dim_idx = 0
        elif self.dimension == "y": dim_idx = 1
        elif self.dimension == "z": dim_idx = 2

        # get values to compare for all objects
        values_to_compare = []
        for argument in self.arguments:
            values_to_compare.append(argument.rot[dim_idx])

        # take the mean absolute deviation and output
        values_to_compare = np.array(values_to_compare)
        return np.abs(values_to_compare - values_to_compare.mean()).mean()

class HaveRotationalDifference_B(ConstraintProposition):
    """A proposition that represents a notion of rotational difference between TWO objects.

    NOTE: This constraint proposition is binary ("_B") for the same reason as HaveTranslationalDifference_B.

    Inherits from the Object abstract class. Difference is defined by the self.dimension attribute
    but only operates over two objects and allows the user to specify how far one wants the objects
    pointed apart with the self.difference attribute. Difference is calculated as obj1 - obj2, so
    to assert that 

    Attributes:
        Superclass attributes.
        target_difference: float, the target difference in the specified location between argument 1 and 2.
        dimension: str, one of "x", "y", or "z", the dimension on which to assert alignment.
    """
    def __init__(self, arguments, target_difference, dimension):
        """Init method."""
        super().__init__(arguments)

        # check to make sure dimension and location are valid
        if dimension not in ["x", "y", "z"]:
            raise ValueError(f"Argument 'dimension' must be one of 'x', 'y', or 'z'.")

        # store attributes
        self.target_difference = target_difference
        self.dimension = dimension

    def define_arity(self):
        """See superclass documentation for details."""
        return 2 # binary

    def badness(self):
        """See superclass documentation for details.
        
        In this case, badness is calculated as the difference between the target and actual differences
        between object 1 and object 2.
        """
        # convert str dimension to index
        dim_idx = 0
        if self.dimension == "x": dim_idx = 0
        elif self.dimension == "y": dim_idx = 1
        elif self.dimension == "z": dim_idx = 2

        # get values to compare for all objects (binary so only two arguments)
        values_to_compare = []
        for argument in self.arguments:
            values_to_compare.append(argument.rot[dim_idx])

        # take differences and output (binary so only two arguments)
        actual_difference = values_to_compare[0] - values_to_compare[1]
        return abs(self.target_difference - actual_difference)

class HaveDistanceBetween_B(ConstraintProposition):
    """A proposition that specifies that the TWO passed arguments must have a certain distance between them.

    NOTE: This constraint proposition is binary ("_B").

    Inherits from the Object abstract class. Distance between is defined as the euclidean distance between
    the pass objects.

    Attributes:
        Superclass attributes.
        target_distance: positive float, the target distance between the two objects.
    """
    def __init__(self, arguments, target_distance):
        """Init method."""
        super().__init__(arguments)

        # check to make sure target_distance is valid and store
        if target_distance < 0:
            raise ValueError(f"Argument 'target_distance' must be >= 0.")
        self.target_distance = target_distance

    def define_arity(self):
        """See superclass documentation for details."""
        return 2 # binary

    def badness(self):
        """See superclass documentation for details.
        
        In this case, badness is calculated as the abs value between the actual distance and target distance.
        """
        actual_distance = np.linalg.norm(self.arguments[0].loc - self.arguments[1].loc)
        return np.abs(self.target_distance - actual_distance)

class IsPointingTowards_U(ConstraintProposition):
    """A proposition that specifies that the ONE passed argument must point towards a point.

    NOTE: This constraint proposition is unary ("_U").

    Inherits from the Object abstract class. Pointing toward a point is defined as having
    the argument object's local x-axis have the same angle as the displacement vector from
    the argument object's location to the target point.

    Attributes:
        Superclass attributes.
        target_point: a list of form [x,y,z] specifying the point to look at.
    """
    def __init__(self, arguments, target_point):
        """Init method."""
        super().__init__(arguments)

        # check to make sure target_point is valid and store
        if len(target_point) != 3:
            raise ValueError(f"Argument 'target_point' must have length 3.")
        self.target_point = target_point

    def define_arity(self):
        """See superclass documentation for details."""
        return 1 # unary

    def badness(self):
        """See superclass documentation for details.
        
        In this case, badness is calculated as 1 - cosine(local_x_axis, displacement)
        where displacement is the vector from the object to the target point.
        """
        # get displacement
        displacement = np.array(self.target_point) - self.arguments[0].loc
        displacement_magnitude = np.linalg.norm(displacement)

        # get local x-axis
        global_x_axis = np.array([1,0,0]) # magnitude 1
        rotation_matrix = R.from_euler(angles=self.arguments[0].rot,
                                       seq="xyz", degrees=True).as_matrix()
        local_x_axis = global_x_axis @ rotation_matrix

        # compute cosine of angle between them with law of cosines
        cosine = np.dot(displacement, local_x_axis) / displacement_magnitude # local_x_axis is unit vector
        return 1 - cosine

class AreSymmetrical_B(ConstraintProposition):
    """A proposition that specifies that the TWO passed arguments must be symmetrical around a point.

    NOTE: This constraint proposition is binary ("_B").

    Inherits from the Object abstract class. Symmetry around a point is defined as having the vector
    from argument_1 to the point be the opposite of the one from argument_2 to the point.

    Attributes:
        Superclass attributes.
        point: a list of form [x,y,z] specifying the point around which the arguments are symmetrical.
    """
    def __init__(self, arguments, point):
        """Init method."""
        super().__init__(arguments)

        # check to make sure point is valid and store
        if len(point) != 3:
            raise ValueError(f"Argument 'point' must have length 3.")
        self.point = point

    def define_arity(self):
        """See superclass documentation for details."""
        return 2 # binary

    def badness(self):
        """See superclass documentation for details.
        
        In this case, badness is calculated as the l1 distance from the sum of the two displacement
        vectors to the zero vector.
        """
        dis1 = np.array(self.point) - self.arguments[0].loc
        dis2 = np.array(self.point) - self.arguments[1].loc
        return np.abs(dis1 + dis2).sum()

class AreNotOverlapping_B(ConstraintProposition):
    """A proposition that specifies that BOTH passed arguments must not overlap.

    NOTE: This constraint proposition is binary ("_B").

    Inherits from the Object abstract class. Overlapping is defined as having the bounding
    boxes returned by the argument objects' get_bounding_intervals overlapping.

    Attributes:
        Superclass attributes.
    """
    def __init__(self, arguments):
        """Init method."""
        super().__init__(arguments)

    def define_arity(self):
        """See superclass documentation for details."""
        return 2 # binary

    def badness(self):
        """See superclass documentation for details.
        
        In this case, badness is calculated as the overlap area if there is an overlap
        and 0 if there is no overlap. The calculations are based on the principle that
        2 axis-aligned boxes (bounding boxes are axis-aligned) overlap if and only if
        the projections to all axes overlap.
        """
        # get bounding intervals (binary so only two arguments)
        [xmin1, xmax1], [ymin1, ymax1], [zmin1, zmax1] = self.arguments[0].get_bounding_intervals()
        [xmin2, xmax2], [ymin2, ymax2], [zmin2, zmax2] = self.arguments[1].get_bounding_intervals()

        # For any axis, we have two bounding intervals for the two arguments, and there are six cases:
        #     1. Interval 1 is strictly before interval 2.
        #     2. Interval 2 is strictly before interval 1.
        #     3. Intervals 1 and 2 overlap but one does not fully contain the other, interval 1 is first.
        #     4. Intervals 1 and 2 overlap but one does not fully contain the other, interval 2 is first.
        #     5. Intervals 1 and 2 overlap and interval 1 fully contains interval 2.
        #     6. Intervals 1 and 2 overlap and interval 2 fully contains interval 1.
        
        # In cases 1 and 2, either (max1 - min2) OR (max2 - min1) will be negative/zero and overlap area is 0.
        # In cases 3 and 4, both (max1 - min2) AND (max2 - min1) will be positive and overlap area is the min between them.
        # In cases 5 and 6, both (max1 - min2) AND (max2 - min1) will be positive and overlap area is the shorter interval.

        # HOWEVER, for cases 5 and 6, if we try to minimize the overlap area, for small steps in either
        # direction, there will not be changes in the badness until the intervals are no longer overlapping.
        # This makes minimizing the overlap area challenging. Therefore, for ease of optimization, it suffices
        # to treat cases 5 & 6 the same as 3 & 4 and let the badness always be whichever one of (max1 - min2)
        # and (max2 - min1) is smaller. In the one-interval-within-another situation of cases 5 and 6, this
        # is equivalent to letting the algorithm push the inner interval in whichever direction is closest
        # to the edge of the outer interval.
        
        if xmax1 - xmin2 <= 0 or xmax2 - xmin1 <= 0:
            x_overlap = 0 # no overlap, cases 1 and 2
        else:
            x_overlap = min(xmax1 - xmin2, xmax2 - xmin1) # has overlap, cases 3 - 6

        if ymax1 - ymin2 <= 0 or ymax2 - ymin1 <= 0:
            y_overlap = 0 # no overlap, cases 1 and 2
        else:
            y_overlap = min(ymax1 - ymin2, ymax2 - ymin1) # has overlap, cases 3 - 6

        if zmax1 - zmin2 <= 0 or zmax2 - zmin1 <= 0:
            z_overlap = 0 # no overlap, cases 1 and 2
        else:
            z_overlap = min(zmax1 - zmin2, zmax2 - zmin1) # has overlap, cases 3 - 6

        total_overlap = x_overlap * y_overlap * z_overlap
        return total_overlap

## Composite constraints:

In [6]:
class IsUpright_U(ConstraintProposition):
    """A unary ("_U") constraint asserting that the passed argument has rotation [0,0,0].
    """
    def __init__(self, arguments):
        super().__init__(arguments)

    def define_arity(self):
        return 1 # unary

    def badness(self):
        return HasRotation_U(arguments=self.arguments, target_rot=[0,0,0]).badness()

class IsAtOrigin_U(ConstraintProposition):
    """A unary ("_U") constraint asserting that the passed argument has location [0,0,0].
    """
    def __init__(self, arguments):
        super().__init__(arguments)

    def define_arity(self):
        return 1 # unary

    def badness(self):
        return HasLocation_U(arguments=self.arguments, target_loc=[0,0,0]).badness()

class AreProximal_B(ConstraintProposition):
    """A binary ("_B") constraint asserting that the TWO passed objects are close but not overlapping.
    """
    def __init__(self, arguments):
        super().__init__(arguments)

    def define_arity(self):
        return 2 # binary

    def badness(self):
        res = (HaveDistanceBetween_B(arguments=self.arguments, target_distance=0).badness()
               + AreNotOverlapping_B(arguments=self.arguments).badness())
        return res

class HaveSameRotation_F(ConstraintProposition):
    """A flexible-arity ("_F") constraint asserting that ALL passed objects must have the same orientation.
    """
    def __init__(self, arguments):
        super().__init__(arguments)

    def define_arity(self):
        return None # flexible-arity

    def badness(self):
        res = (AreRotationallyAligned_F(arguments=self.arguments, dimension="x").badness()
               + AreRotationallyAligned_F(arguments=self.arguments, dimension="y").badness()
               + AreRotationallyAligned_F(arguments=self.arguments, dimension="z").badness())
        return res

class AreTopAligned_F(ConstraintProposition):
    """A flexible-arity ("_F") constraint asserting that ALL passed objects are top-aligned along the z axis.
    """
    def __init__(self, arguments):
        super().__init__(arguments)

    def define_arity(self):
        return None # flexible-arity

    def badness(self):
        return AreTranslationallyAligned_F(arguments=self.arguments, dimension="z",
                                           location="bounding_max").badness()

class AreBottomAligned_F(ConstraintProposition):
    """A flexible-arity ("_F") constraint asserting that ALL passed objects are bottom-aligned along the z axis.
    """
    def __init__(self, arguments):
        super().__init__(arguments)

    def define_arity(self):
        return None # flexible-arity

    def badness(self):
        return AreTranslationallyAligned_F(arguments=self.arguments, dimension="z",
                                           location="bounding_min").badness()

class AreXPlusAligned_F(ConstraintProposition):
    """A flexible-arity ("_F") constraint asserting that ALL passed objects are aligned along the positive x axis.
    """
    def __init__(self, arguments):
        super().__init__(arguments)

    def define_arity(self):
        return None # flexible-arity

    def badness(self):
        return AreTranslationallyAligned_F(arguments=self.arguments, dimension="x",
                                           location="bounding_max").badness()

class AreXMinusAligned_F(ConstraintProposition):
    """A flexible-arity ("_F") constraint asserting that ALL passed objects are aligned along the negative x axis.
    """
    def __init__(self, arguments):
        super().__init__(arguments)

    def define_arity(self):
        return None # flexible-arity

    def badness(self):
        return AreTranslationallyAligned_F(arguments=self.arguments, dimension="x",
                                           location="bounding_min").badness()

class AreYPlusAligned_F(ConstraintProposition):
    """A flexible-arity ("_F") constraint asserting that ALL passed objects are aligned along the positive y axis.
    """
    def __init__(self, arguments):
        super().__init__(arguments)

    def define_arity(self):
        return None # flexible-arity

    def badness(self):
        return AreTranslationallyAligned_F(arguments=self.arguments, dimension="y",
                                           location="bounding_max").badness()

class AreYMinusAligned_F(ConstraintProposition):
    """A flexible-arity ("_F") constraint asserting that ALL passed objects are aligned along the negative y axis.
    """
    def __init__(self, arguments):
        super().__init__(arguments)

    def define_arity(self):
        return None # flexible-arity

    def badness(self):
        return AreTranslationallyAligned_F(arguments=self.arguments, dimension="y",
                                           location="bounding_min").badness()

class AreSymmetricalAround_T(ConstraintProposition):
    """A ternary ("_T") constraint asserting that the passed objects must look like:
    argument_0   <---equal--->   argument_1   <---equal--->   argument_2
    """
    def __init__(self, arguments):
        super().__init__(arguments)

    def define_arity(self):
        return 3 # ternary

    def badness(self):
        return AreSymmetrical_B(arguments=[self.arguments[0],self.arguments[2]],
                                point=self.arguments[1].loc).badness()

# overlap defined above in primitive constraints

class AreParallelX_B(ConstraintProposition):
    """A binary ("_B") constraint asserting that the TWO passed objects must be parallel in x-rotation.
    """
    def __init__(self, arguments):
        super().__init__(arguments)

    def define_arity(self):
        return 2 # binary

    def badness(self):
        return AreRotationallyAligned_F(arguments=self.arguments, dimension="x").badness()

class AreParallelY_B(ConstraintProposition):
    """A binary ("_B") constraint asserting that the TWO passed objects must be parallel in y-rotation.
    """
    def __init__(self, arguments):
        super().__init__(arguments)

    def define_arity(self):
        return 2 # binary

    def badness(self):
        return AreRotationallyAligned_F(arguments=self.arguments, dimension="y").badness()

class AreParallelZ_B(ConstraintProposition):
    """A binary ("_B") constraint asserting that the TWO passed objects must be parallel in z-rotation.
    """
    def __init__(self, arguments):
        super().__init__(arguments)

    def define_arity(self):
        return 2 # binary

    def badness(self):
        return AreRotationallyAligned_F(arguments=self.arguments, dimension="z").badness()

class ArePerpendicularX_B(ConstraintProposition):
    """A binary ("_B") constraint asserting that the TWO passed objects must be perpendicular in x-rotation.
    """
    def __init__(self, arguments):
        super().__init__(arguments)

    def define_arity(self):
        return 2 # binary

    def badness(self):
        return HaveRotationalDifference_B(self.arguments, target_difference=90,
                                          dimension="x").badness()

class ArePerpendicularY_B(ConstraintProposition):
    """A binary ("_B") constraint asserting that the TWO passed objects must be perpendicular in y-rotation.
    """
    def __init__(self, arguments):
        super().__init__(arguments)

    def define_arity(self):
        return 2 # binary

    def badness(self):
        return HaveRotationalDifference_B(self.arguments, target_difference=90,
                                          dimension="y").badness()

class ArePerpendicularZ_B(ConstraintProposition):
    """A binary ("_B") constraint asserting that the TWO passed objects must be perpendicular in z-rotation.
    """
    def __init__(self, arguments):
        super().__init__(arguments)

    def define_arity(self):
        return 2 # binary

    def badness(self):
        return HaveRotationalDifference_B(self.arguments, target_difference=90,
                                          dimension="z").badness()

# Main problem class:

In [7]:
class Problem:
    """The primary way users interact with the constraint solver.

    This class handles problem creation, adding optimizable objects and constraint propositions, solving
    the problem, and plotting the current state of all objects. Every optimizable object currently
    in the problem is given an id by the user code on addition. Any object not added to the problem
    is treated as constant and immovable.

    NOTE: When adding constraint propositions between optimizable objects, it is recommended to use the
          function get_optimizable_object with the desired object's id to prevent confusion. Any objects
          that are not explicitly id-ed and added through add_optimizable_object but are mentioned in
          any of the constraint propositions is treated as immovable.

    Attributes:
        optimizable_objects: A dictionary of GeometricObjects that are optimizable, each with a unique id.
        constraint_propositions: A list of ConstraintPropositions between GeometricObjects that we want to satisfy.
        constraint_weights: A list of floats representing the weight of each constraint proposition at the same idx.
    """
    def __init__(self):
        """Create an empty problem."""
        self.optimizable_objects = {}
        self.constraint_propositions = []
        self.constraint_weights = []

    def add_optimizable_object(self, object, id):
        """Add a GeometricObject to self.optimizable_objects with given id.
        Args:
            object: A GeometricObject to be added.
            id: The id to identify the object with.
        Returns:
            None.
        Raises:
            ValueError if an object with id is already in self.optimizable_objects.
        """
        if id in self.optimizable_objects:
            raise ValueError(f"The id passed ({id}) is already associated with an object in the problem.")
        self.optimizable_objects[id] = object

    def remove_optimizable_object(self, id):
        """Remove an GeometricObject from self.optimizable_objects by id.
        Args:
            id: The id to of the object to remove.
        Returns:
            None.
        Raises:
            ValueError if id passed is not present in self.optimizable_objects.
        """
        if id not in self.optimizable_objects:
            raise ValueError(f"The id passed ({id}) is not associated with any object in the problem.")
        self.optimizable_objects.pop(id)

    def get_optimizable_object(self, id):
        """Get pointer to an GeometricObject from self.optimizable_objects by id.
        Args:
            id: The id to of the object to get.
        Returns:
            A pointer to the desired object.
        Raises:
            ValueError if id passed is not present in self.optimizable_objects.
        """
        if id not in self.optimizable_objects:
            raise ValueError(f"The id passed ({id}) is not associated with any object in the problem.")
        return self.optimizable_objects[id]

    def add_constraint_proposition(self, proposition, weight):
        """Add a new ConstraintProposition to the problem.
        Args:
            proposition: The constraint proposition to add.
            weight: The weight of this constraint proposition.
        Returns:
            None.
        """
        self.constraint_propositions.append(proposition)
        self.constraint_weights.append(weight)

    def _flatten_optimizable_parameters(self):
        """Helper function to flatten the optimizable parameters of all optimizable objects for scipy."""
        arrays = []
        for id, object in self.optimizable_objects.items():
            arrays.append(object.get_optimizable_attr())
        return np.concatenate(arrays)

    def _recover_optimizable_parameters(self, flat_array):
        """Helper function to recover the optimizable parameters of all optimizable objects given flat array.
        NOTE: flat_array has to be one originally outputted by _flatten_optimizable_parameters."""
        obj_len = len(next(iter(self.optimizable_objects.values())).get_optimizable_attr())
        cur = 0
        for id, object in self.optimizable_objects.items():
            object.set_optimizable_attr(flat_array[cur:cur+obj_len])
            cur = cur + obj_len

    def solve(self, verbose=False):
        """Tune the optimizable parameters of all optimizable objects to best satisfy constraint propositions.
    
        This is the main function of the solver. It modifies objects in place.
    
        Args:
            verbose: bool, whether or not to print full scipy.minimize results.
        Returns:
            None. Modifies self.optimizable_objects in-place.
        """
        def objective(flat_array):
            # update the objects
            self._recover_optimizable_parameters(flat_array)

            # compute the total weighted badness measure for all constraint propositions
            total_badness = 0
            for i in range(len(self.constraint_propositions)):
                total_badness += (self.constraint_propositions[i].badness() * self.constraint_weights[i])
            return total_badness

        # compute solution, changing objects every iteration along the way
        # solution = minimize(objective, x0=self._flatten_optimizable_parameters(), method="Nelder-Mead")
        solution = minimize(objective, x0=self._flatten_optimizable_parameters(), method="powell")

        # optionally print full optimization results
        if verbose:
            print(solution)

        # set the objects to the final solution
        self._recover_optimizable_parameters(solution.x)

    def plot_on_ax(self, ax, ax_title, fixed_axes=None, elev=30, azim=40):
        """Plot the problem with all optimizable objects on axis ax.

        This generates a 3D plot.

        Args:
            ax: matplotlib axes to plot on.
            ax_title: str, title of axes.
            fixed_axes: If not None, all x/y/z-lims will be set to (-fixed_axes,fixed_axes).
            elev, azim: matplotlib 3D plot viewing angle.
        Returns:
            None. Modifies argument ax.
        """
        # define colors
        colors = list(mcolors.TABLEAU_COLORS.keys())

        # add all objects to ax
        legend_patchs = []
        color_idx = 0 # will wrap around
        for id, object in self.optimizable_objects.items(): 
            legend_patchs.append(object.add_self_to_axis(ax, label=id, color=colors[color_idx]))
            color_idx += 1
            color_idx = color_idx % len(colors)

        # configure plot info
        ax.set_title(ax_title, y=0.95)
        ax.set_xlabel("x")
        ax.set_ylabel("y")
        ax.set_zlabel("z")
        ax.legend(handles=legend_patchs, loc="upper right", bbox_to_anchor=(1.32,1))
        if fixed_axes is not None:
            ax.set_xlim(-fixed_axes,fixed_axes)
            ax.set_ylim(-fixed_axes,fixed_axes)
            ax.set_zlim(-fixed_axes,fixed_axes)

        #configure view
        ax.set_proj_type("persp",focal_length=0.2)
        ax.view_init(elev=elev,azim=azim)

# Data Generation:

In [8]:
def generate_data(data_directory, run_name, num_datapoints, axes_scale=10,
                  min_object_count=3, max_object_count=10,
                  min_constraint_count=5, max_constraint_count=15,
                  save_visualizations=False):
    """Generate data mapping from initial objects and a list of constraint propositions to final/solved objects.

    This function will create a new pickle file with name "{run_name}.pkl" in data_directory. If
    "{run_name}.pkl" already exists, it will be overriden. Unpacking "{run_name}.pkl"
    will yield a list of dictionaries (each datapoint is represented as a dictionary).

    NOTE: Current version of function only generates data where x and y rotation are both 0.

    Args:
        data_directory: str, directory in which to create a new folder to store generated data.
        run_name: str, the name of the pkl file to create will be "{run_name}.pkl".
        num_datapoints: int, number of datapoints to generate.
        axes_scale: int, the scale of the coordinates to generate, axes will range [-axes_scale, axes_scale].
        {min,max}_object_count: ints, the min/max number of objects in each datapoint (sampled uniformally).
        {min,max}_constraint_count: ints, the min/max number of constraints in each datapoint (sampled uniformally).

    Returns:
        None.
    """
    # define function to convert object to dictionary according to given specifications
    def obj_to_dict(obj, name):
        obj_dict = {
            "name": name,
            "position": obj.loc,
            "size": obj.scale,
            "orientation": obj.rot
        }
        return obj_dict

    # define function to convert constraint to dictionary according to given specifications
    def constraint_to_dict(name, args, weight):
        obj_dict = {
            "class": name,
            "args": args,
            "weight": weight
        }
        return obj_dict
    
    # define meta-configurations and all available constraints
    object_min_scale = 1
    object_max_scale = 3
    object_coord_range = axes_scale - object_max_scale/2 # actual range is [-object_coord_range, object_coord_range]
    constraints = [IsUpright_U, IsAtOrigin_U, AreProximal_B, HaveSameRotation_F,
                   AreTopAligned_F, AreBottomAligned_F, AreSymmetricalAround_T,
                   AreNotOverlapping_B, AreParallelZ_B, ArePerpendicularZ_B,
                   AreXPlusAligned_F, AreXMinusAligned_F, AreYPlusAligned_F, AreYMinusAligned_F]
    constraint_names = ["IsUpright", "IsAtOrigin", "AreProximal", "HaveSameRotation",
                        "AreTopAligned", "AreBottomAligned", "AreSymmetricalAround",
                        "AreNotOverlapping", "AreParallelZ", "ArePerpendicularZ",
                        "AreXPlusAligned", "AreXMinusAligned", "AreYPlusAligned", "AreYMinusAligned"]
    constraint_arities = [1, 1, 2, None, None, None, 3, 2, 2, 2, None, None, None, None]
    constraint_weights = np.array([0.5, 0.5, 2, 1, 1, 1, 1, 2, 1, 1, 0.5, 0.5, 0.5, 0.5])
    constraint_weights = constraint_weights / constraint_weights.sum()

    # optionally setup folder to save figures in
    if save_visualizations:
        figures_folder_name = os.path.join(data_directory, f"{run_name}_visualizations/")
        if os.path.exists(figures_folder_name):
            shutil.rmtree(figures_folder_name)
        os.makedirs(figures_folder_name)

    # create holder for final dataset to pickle
    dataset = []
    
    # main datapoint generation loop
    for datapoint_id in tqdm(range(num_datapoints)):
        # create holders to be later put into a dictionary to output
        initial_objs_output = []
        constraints_output = []
        solved_objects_output = []
        
        # initialize problem and determine how many objects and contraints this datapoint will have
        problem = Problem()
        num_objs = np.random.randint(low=min_object_count, high=max_object_count+1)
        num_constraints = np.random.randint(low=min_constraint_count, high=max_constraint_count+1)  

        # create the objects (random initialization, 50% of objects will be upright)
        objs = []
        obj_names = []
        for i in range(1,num_objs+1):
            #create random object
            obj = Cuboid(loc=np.random.uniform(low=-object_coord_range ,high=object_coord_range, size=(3)),
                         rot=[0,0,0 if np.random.uniform() < 0.5 else np.random.uniform(low=0, high=360)],
                         scale=np.random.uniform(low=object_min_scale ,high=object_max_scale, size=(3)))
            # name and add to lists and problem
            obj_name = f"Cube{i}"
            objs.append(obj)
            obj_names.append(obj_name)
            problem.add_optimizable_object(obj, obj_name)
            # add in dictionary form to initial_objs_output
            initial_objs_output.append(obj_to_dict(obj, obj_name))
        objs = np.array(objs)
        obj_names = np.array(obj_names)

        # optionally setup plot
        if save_visualizations:
            fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(8,12), subplot_kw={"projection": "3d"})
            problem.plot_on_ax(ax=ax[0], ax_title="Initial Objects (Unconstrained)", fixed_axes=axes_scale)

        # add the constraints (between random objects)
        if save_visualizations: # optionally create a string to put at bottom of visualization
            description = ""
        for i in range(1,num_constraints+1):
            # find random constraint
            constraint_idx = np.random.choice(len(constraints), p=constraint_weights) # [0,len(constraints))
            constraint = constraints[constraint_idx]
            constraint_name = constraint_names[constraint_idx]
            constraint_arity = constraint_arities[constraint_idx]
            if constraint_arity is None: # for flexible arity constraints
                constraint_arity = np.random.randint(low=1, high=num_objs)

            # find random objects as arguments and find random weight
            constraint_arg_idx = np.random.choice(a=num_objs, size=constraint_arity, replace=False)
            constraint_args = objs[constraint_arg_idx]
            constraint_arg_names = obj_names[constraint_arg_idx]
            constraint_weight = np.random.choice([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]) # random weight

            # add to problem, constraints_output, and optionally update description
            problem.add_constraint_proposition(constraint(arguments=constraint_args),
                                               weight=constraint_weight)
            constraints_output.append(constraint_to_dict(name=constraint_name,
                                                         args=list(constraint_arg_names),
                                                         weight=constraint_weight))
            if save_visualizations:
                description += f"Constraint {i} (with weight {constraint_weight}): {constraint_name}({constraint_arg_names}).\n"

        # solve the problem
        problem.solve()

        # add the solved objects to solved_objects_output
        for i in range(num_objs):
            solved_objects_output.append(obj_to_dict(objs[i], obj_names[i]))

        # optionally plot the final state and save figure
        if save_visualizations:
            problem.plot_on_ax(ax=ax[1], ax_title="Solved Objects (Constrained)", fixed_axes=axes_scale)
            file_name = os.path.join(figures_folder_name, f"visualization_{datapoint_id}.png")
            plt.figtext(x=0,y=0.39,s=description)
            plt.subplots_adjust(hspace=1)
            plt.savefig(file_name)
            plt.close()

        # create dictionary for this datapoint and add to dataset
        datapoint = {
            "initial_objects":initial_objs_output,
            "constraints":constraints_output,
            "solved_objects":solved_objects_output
        }
        dataset.append(datapoint)

    # pickle dataset
    pkl_path = os.path.join(data_directory, f"{run_name}.pkl")
    with open(pkl_path, "wb") as f:
        pickle.dump(dataset, f)

# generate_data(data_directory="./GeneratedData", run_name="Dataset8000",
#               num_datapoints=8000, axes_scale=10,
#               min_object_count=3, max_object_count=10,
#               min_constraint_count=5, max_constraint_count=15,
#               save_visualizations=True);

generate_data(data_directory="./GeneratedData", run_name="Dataset2",
              num_datapoints=2, axes_scale=10,
              min_object_count=3, max_object_count=10,
              min_constraint_count=5, max_constraint_count=15,
              save_visualizations=True);

100%|██████████| 2/2 [00:01<00:00,  1.11it/s]


# Data evaluation:

Given a list of object dictionaries in pickled format and a list of constraint dictionaries, output the sum of the badness.

In [9]:
def evaluate(objs_dict_list, constraints_dict_list):
    name_to_constraint = {
        "IsUpright":IsUpright_U,
        "IsAtOrigin":IsAtOrigin_U,
        "AreProximal":AreProximal_B,
        "HaveSameRotation":HaveSameRotation_F,
        "AreTopAligned":AreTopAligned_F,
        "AreBottomAligned":AreBottomAligned_F,
        "AreSymmetricalAround":AreSymmetricalAround_T,
        "AreNotOverlapping":AreNotOverlapping_B,
        "AreParallelZ":AreParallelZ_B,
        "ArePerpendicularZ":ArePerpendicularZ_B,
        "AreXPlusAligned":AreXPlusAligned_F,
        "AreXMinusAligned":AreXMinusAligned_F,
        "AreYPlusAligned":AreYPlusAligned_F,
        "AreYMinusAligned":AreYMinusAligned_F
    }
    def dict_to_obj(dict):
        name = dict["name"]
        obj = Cuboid(loc=dict["position"], rot=dict["orientation"], scale=dict["size"])
        return obj, name
    def dict_list_to_name_obj_map(dict_list):
        name_obj_map = {}
        for dict in dict_list:
            obj, name = dict_to_obj(dict)
            name_obj_map[name] = obj
        return name_obj_map
    def dict_to_constraint(dict, name_obj_map):
        arguments = []
        for arg_name in dict["args"]:
            arguments.append(name_obj_map[arg_name])
        constraint = name_to_constraint[dict["class"]](arguments)
        weight = dict["weight"]
        return constraint, weight

    name_obj_map = dict_list_to_name_obj_map(objs_dict_list)
    total_badness = 0
    for constraint_dict in constraints_dict_list:
        constraint, weight = dict_to_constraint(constraint_dict, name_obj_map)
        total_badness += constraint.badness() * weight

    return total_badness

In [11]:
with open("./GeneratedData/Dataset2.pkl", "rb") as f:
        dataset = pickle.load(f)
datapoint = 1
print(evaluate(dataset[datapoint]["initial_objects"], dataset[datapoint]["constraints"]))
print(evaluate(dataset[datapoint]["solved_objects"], dataset[datapoint]["constraints"]))

31.851796333143337
15.106922222983492
